##Projet de Mastercamp

In [1]:
import pandas as pd
import numpy as np

In [2]:
valeur = pd.read_csv(r"valeursfoncieres-2023.txt", delimiter="|")

C:\Users\q5685\AppData\Local\Temp\ipykernel_8272\3540214829.py:1: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  valeur = pd.read_csv(r"valeursfoncieres-2023.txt", delimiter="|")


In [3]:
print(valeur.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 43 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Identifiant de document     float64
 1   Reference document          float64
 2   1 Articles CGI              float64
 3   2 Articles CGI              float64
 4   3 Articles CGI              float64
 5   4 Articles CGI              float64
 6   5 Articles CGI              float64
 7   No disposition              int64  
 8   Date mutation               object 
 9   Nature mutation             object 
 10  Valeur fonciere             object 
 11  No voie                     float64
 12  B/T/Q                       object 
 13  Type de voie                object 
 14  Code voie                   object 
 15  Voie                        object 
 16  Code postal                 float64
 17  Commune                     object 
 18  Code departement            object 
 19  Code commune         

In [4]:
valeur_in_cleaning = (valeur.copy())
valeur_in_cleaning.dropna(axis=1, how="all", inplace=True)
print(valeur_in_cleaning.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 35 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   No disposition              int64  
 1   Date mutation               object 
 2   Nature mutation             object 
 3   Valeur fonciere             object 
 4   No voie                     float64
 5   B/T/Q                       object 
 6   Type de voie                object 
 7   Code voie                   object 
 8   Voie                        object 
 9   Code postal                 float64
 10  Commune                     object 
 11  Code departement            object 
 12  Code commune                int64  
 13  Prefixe de section          float64
 14  Section                     object 
 15  No plan                     int64  
 16  No Volume                   object 
 17  1er lot                     object 
 18  Surface Carrez du 1er lot   object 
 19  2eme lot             


#list_surface_carrez_lot = ["Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]
#valeur_in_cleaning["Surface Carrez Lots Total"] = 0
#print(valeur_in_cleaning[[list_surface_carrez_lot[0]]])

for i  in list_surface_carrez_lot:
    valeur_in_cleaning[valeur_in_cleaning[i].isna()] = 0

valeur_in_cleaning.convert_dtypes(convert_floating=True)

print(valeur_in_cleaning.info())

for i  in list_surface_carrez_lot:
    valeur_in_cleaning["Surface Carrez Lots Total"] = valeur_in_cleaning["Surface Carrez Lots Total"] + temp_2
print(valeur_in_cleaning[list_surface_carrez_lot])

#print(valeur_in_cleaning["Surface Carrez Lots Total"])
#valeur_in_cleaning["Surface Carrez Lots Total"] =  valeur_in_cleaning[["Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]].sum()

On regarde maintenenant les colonnes qui qui possèdes des valeur et que l'on veut garder
Pour cela, on va regarder les colonnes restanes et voir le pour centage de ligne remplis
On récupère tout les colonnes dont moint de 10 % des lignes sont remplis. On va aussi regarder si une colonne comprenant la somme des entrées dans les colonnes sur las surface carrés des lots.

In [5]:


index_restant = list(valeur_in_cleaning.columns)
print(index_restant)

insignificante_column = list()




for i in index_restant:
    percentage_use = len((valeur_in_cleaning[i])[valeur_in_cleaning[i].notna()])/len(valeur_in_cleaning[i])
    print(f"Pour {i}, le pourcentage de ligne remplis est de {percentage_use}")
    if percentage_use < 0.1:
        insignificante_column.append(i)



mask = valeur_in_cleaning["Surface Carrez du 1er lot"].notna() | valeur_in_cleaning["Surface Carrez du 2eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 3eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 4eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 5eme lot"].notna()

nb_nn = len(valeur_in_cleaning[mask]["Surface Carrez du 4eme lot"])
print(f"Pour la somme des surface carrez des metros, le pourcentage de ligne remplis est de {nb_nn/valeur_in_cleaning.shape[0]}")

['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere', 'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune', 'Prefixe de section', 'Section', 'No plan', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Nombre de lots', 'Code type local', 'Type local', 'Surface reelle bati', 'Nombre pieces principales', 'Nature culture', 'Nature culture speciale', 'Surface terrain']
Pour No disposition, le pourcentage de ligne remplis est de 1.0
Pour Date mutation, le pourcentage de ligne remplis est de 1.0
Pour Nature mutation, le pourcentage de ligne remplis est de 1.0
Pour Valeur fonciere, le pourcentage de ligne remplis est de 0.9913123271903311
Pour No voie, le pourcentage de ligne remplis est de 0.6201649850214851
Pour B/T/Q, le pourcentage de l

On voit donc q'une colonne contennat la somme des surfac ecarrés de chaque lots serait remplit à 11,9 % donc on vas crée un nouvelle colonnes.
Pour cela on vas d'abord convertir c'est colonne en float :
 - On remplace les NaN par des 0
 - On convertit  les colonnes en entier
 - On fait la somme qui est stocké dans une nouvelle colonne

In [6]:

valeur_in_cleaning.fillna(0, inplace=True)

print(valeur_in_cleaning)



#valeur_in_cleaning["Surface Carrez lot total"] = valeur_in_cleaning[["Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]].sum(axis=0)

         No disposition Date mutation Nature mutation Valeur fonciere  \
0                     1    05/01/2023           Vente      1070000,00   
1                     1    05/01/2023           Vente      1070000,00   
2                     1    05/01/2023           Vente      1070000,00   
3                     1    03/01/2023           Vente       152200,00   
4                     1    05/01/2023           Vente       269000,00   
...                 ...           ...             ...             ...   
3727000               1    05/10/2023    Adjudication      3931000,00   
3727001               1    05/10/2023    Adjudication      3931000,00   
3727002               1    05/10/2023    Adjudication      3931000,00   
3727003               1    28/12/2023           Vente        98250,00   
3727004               1    28/12/2023           Vente        98250,00   

         No voie B/T/Q Type de voie Code voie               Voie  Code postal  \
0          184.0     0          ALL      0

On enlève convertit les colonne sous forme d'entier

In [8]:
liste_des_lot = ["Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]

print(valeur_in_cleaning)

for i in liste_des_lot:
    print(valeur_in_cleaning[i])
    valeur_in_cleaning[i] = valeur_in_cleaning[i].replace(',', '.', regex=True)
    valeur_in_cleaning[i] = (pd.to_numeric(valeur_in_cleaning[i])).astype("Float64")
    print(valeur_in_cleaning[i])

         No disposition Date mutation Nature mutation Valeur fonciere  \
0                     1    05/01/2023           Vente      1070000,00   
1                     1    05/01/2023           Vente      1070000,00   
2                     1    05/01/2023           Vente      1070000,00   
3                     1    03/01/2023           Vente       152200,00   
4                     1    05/01/2023           Vente       269000,00   
...                 ...           ...             ...             ...   
3727000               1    05/10/2023    Adjudication      3931000,00   
3727001               1    05/10/2023    Adjudication      3931000,00   
3727002               1    05/10/2023    Adjudication      3931000,00   
3727003               1    28/12/2023           Vente        98250,00   
3727004               1    28/12/2023           Vente        98250,00   

         No voie B/T/Q Type de voie Code voie               Voie  Code postal  \
0          184.0     0          ALL      0

On crée la somme une nouvelle colonne qui la somme des surface carrez des lots

In [9]:

valeur_in_cleaning["Surface Carrez Totale des Lots"] = valeur_in_cleaning[liste_des_lot].sum(axis=1)

print(len(valeur_in_cleaning[valeur_in_cleaning["Surface Carrez Totale des Lots"] != 0.0]["Surface Carrez Totale des Lots"])/len(valeur_in_cleaning["Surface Carrez Totale des Lots"]))

0.11958449210559148


On enlève toutes les données dont le pourcentage est inférieur à 10 %.

In [11]:
#print(valeur_in_cleaning.drop(labels=insignificante_column, axis=1))
print("On va enlever : ")
for i in insignificante_column:
    print(f"- {i}")
valeur_in_cleaning.drop(labels=insignificante_column, axis=1, inplace=True).info()
print(valeur_in_cleaning)

On va enlever : 
- B/T/Q
- Prefixe de section
- No Volume
- Surface Carrez du 1er lot
- 2eme lot
- Surface Carrez du 2eme lot
- 3eme lot
- Surface Carrez du 3eme lot
- 4eme lot
- Surface Carrez du 4eme lot
- 5eme lot
- Surface Carrez du 5eme lot
- Nature culture speciale


KeyError: "['B/T/Q', 'Prefixe de section', 'No Volume', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Nature culture speciale'] not found in axis"

Sure les lignes restantes, nous allons enlever tous les numéros d'identifications

In [12]:
valeur_in_cleaning.drop(labels=["No disposition", "No voie", "Voie",  "No plan"], axis=1, inplace=True)

print(valeur_in_cleaning.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 19 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Date mutation                   object 
 1   Nature mutation                 object 
 2   Valeur fonciere                 object 
 3   Type de voie                    object 
 4   Code voie                       object 
 5   Code postal                     float64
 6   Commune                         object 
 7   Code departement                object 
 8   Code commune                    int64  
 9   Section                         object 
 10  1er lot                         object 
 11  Nombre de lots                  int64  
 12  Code type local                 float64
 13  Type local                      object 
 14  Surface reelle bati             float64
 15  Nombre pieces principales       float64
 16  Nature culture                  object 
 17  Surface terrain            